In [97]:
import pandas as pd
import re
import string

#NLP
import nltk
from nltk.corpus import stopwords
import re
import string

import time
import warnings
warnings.filterwarnings('ignore')

In [78]:
true = pd.read_csv('Datasets/Single Source/Non_treated/True.csv')
fake = pd.read_csv('Datasets/Single Source/Non_treated/Fake.csv')

In [79]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [80]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [81]:
true.shape, fake.shape

((21417, 4), (23481, 4))

In [82]:
true['target'] = 1
fake['target'] = 0

In [83]:
unknown_publishers = []
for index,row in enumerate(true['text'].values):
    try:
        record = row.split(" -", maxsplit=1)
        #if no text part is present, following will give error
        record[1]
        #if len of piblication part is greater than 260
        #following will give error, ensuring no text having "-" in between is counted
        assert(len(record[0]) < 260)
    except:
        unknown_publishers.append(index)

In [84]:
#Seperating Publication info, from actual text
publisher = []
tmp_text = []
for index,row in enumerate(true['text'].values):
    if index in unknown_publishers:
        #Add unknown of publisher not mentioned
        tmp_text.append(row)
        
        publisher.append("Unknown")
        continue
    record = row.split(" -", maxsplit=1)
    publisher.append(record[0])
    tmp_text.append(record[1])
    
true["publisher"] = publisher
true["text"] = tmp_text

del publisher, tmp_text, record, unknown_publishers

In [85]:
true.sample(4)

,title,text,subject,date,target,publisher
13433,Mexico ruling party hopeful 14 points adrift i...,The newly unveiled presidential contender of ...,worldnews,"November 30, 2017",1,MEXICO CITY (Reuters)
2900,Illinois risks rating cut to junk even with bu...,Even if Illinois’ House of Representatives ta...,politicsNews,"July 5, 2017",1,CHICAGO (Reuters)
5322,Trump says he'll decide on national security a...,President Donald Trump will interview four ca...,politicsNews,"February 18, 2017",1,(This story corrects year to 2001 in paragrap...
21181,Austrian Chancellor's party sues Foreign Minis...,Austria s ruling Social Democratic Party (SPO...,worldnews,"September 1, 2017",1,VIENNA (Reuters)


In [86]:
#checking for rows with empty text like row:8970
[index for index,text in enumerate(true['text'].values) if str(text).strip() == '']

[8970]

In [87]:
#checking for rows with empty text like row:8970
[index for index,text in enumerate(fake['text'].values) if str(text).strip() == '']

[10923,
 11041,
 11190,
 11225,
 11236,
 11241,
 11247,
 11249,
 11267,
 11268,
 11283,
 11284,
 11285,
 11289,
 11290,
 11292,
 11295,
 11296,
 11301,
 11303,
 11304,
 11305,
 11309,
 11314,
 11317,
 11319,
 11322,
 11330,
 11334,
 11335,
 11340,
 11343,
 11348,
 11351,
 11352,
 11357,
 11371,
 11373,
 11374,
 11382,
 11397,
 11402,
 11403,
 11409,
 11410,
 11412,
 11415,
 11419,
 11421,
 11427,
 11431,
 11432,
 11440,
 11448,
 11450,
 11453,
 11462,
 11464,
 11465,
 11472,
 11473,
 11475,
 11478,
 11489,
 11491,
 11493,
 11494,
 11501,
 11505,
 11507,
 11520,
 11531,
 11532,
 11533,
 11538,
 11542,
 11547,
 11549,
 11551,
 11559,
 11563,
 11581,
 11585,
 11589,
 11590,
 11614,
 11624,
 11625,
 11627,
 11631,
 11636,
 11637,
 11643,
 11650,
 11658,
 11661,
 11672,
 11679,
 11681,
 11684,
 11686,
 11688,
 11692,
 11708,
 11718,
 11729,
 11739,
 11753,
 11765,
 11768,
 11777,
 11782,
 11786,
 11788,
 11792,
 11793,
 11803,
 11806,
 11813,
 11821,
 11831,
 11832,
 11841,
 11844,
 11848,


In [88]:
#dropping this record
true = true.drop(8970, axis=0)

In [89]:
# checking for the same in fake news
empty_fake_index = [index for index,text in enumerate(fake.text.values) if str(text).strip() == '']
print(f"No of empty rows: {len(empty_fake_index)}")
fake.iloc[empty_fake_index].tail()

No of empty rows: 630


,title,text,subject,date,target
21816,BALTIMORE BURNS: MARYLAND GOVERNOR BRINGS IN N...,,left-news,"Apr 27, 2015",0
21826,FULL VIDEO: THE BLOCKBUSTER INVESTIGATION INTO...,,left-news,"Apr 25, 2015",0
21827,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,left-news,"Apr 25, 2015",0
21857,(VIDEO)ICE PROTECTING OBAMA: WON’T RELEASE NAM...,,left-news,"Apr 14, 2015",0
21873,(VIDEO) HYSTERICAL SNL TAKE ON HILLARY’S ANNOU...,,left-news,"Apr 12, 2015",0


# Preprocessing

In [90]:
#Combining Title and Text
true["text"] = true["title"] + " " + true["text"]
fake["text"] = fake["title"] + " " + fake["text"]

In [91]:
def clean_text(text):
    
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

true['text'] = true['text'].apply(lambda x : clean_text(x))
fake['text'] = fake['text'].apply(lambda x : clean_text(x))

In [95]:
#Tokenization

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

true['text'] = true['text'].apply(lambda x : tokenizer.tokenize(x))
fake['text'] = fake['text'].apply(lambda x : tokenizer.tokenize(x))

In [102]:
#THIS TAKES THE MOST TIME.

t1 = time.time()

i = 0
def remove_stopwords(text):
    global i
    words = [w for w in text if w not in stopwords.words('english')]
    i+=1
    
    if(i%1000==0):
        print("Processed ",i," rows")
    
    return words

print("\nTRUE NEWS")
true['text'] = true['text'].apply(lambda x : remove_stopwords(x))
print("\nFAKE NEWS")
fake['text'] = fake['text'].apply(lambda x : remove_stopwords(x))

t2 = time.time()

print("\nTotal Time taken ", t2-t1)


TRUE NEWS
Processed  1000  rows
Processed  2000  rows
Processed  3000  rows
Processed  4000  rows
Processed  5000  rows
Processed  6000  rows
Processed  7000  rows
Processed  8000  rows
Processed  9000  rows
Processed  10000  rows
Processed  11000  rows
Processed  12000  rows
Processed  13000  rows
Processed  14000  rows
Processed  15000  rows
Processed  16000  rows
Processed  17000  rows
Processed  18000  rows
Processed  19000  rows
Processed  20000  rows
Processed  21000  rows

FAKE NEWS
Processed  22000  rows
Processed  23000  rows
Processed  24000  rows
Processed  25000  rows
Processed  26000  rows
Processed  27000  rows
Processed  28000  rows
Processed  29000  rows
Processed  30000  rows
Processed  31000  rows
Processed  32000  rows
Processed  33000  rows
Processed  34000  rows
Processed  35000  rows
Processed  36000  rows
Processed  37000  rows
Processed  38000  rows
Processed  39000  rows
Processed  40000  rows
Processed  41000  rows
Processed  42000  rows
Processed  43000  row

In [103]:
#Token normalization, for easier loading of csv next time

def combine_text(listoftext):
    
    combined_text = ' '.join(listoftext)
    return combined_text

true['text'] = true['text'].apply(lambda x : combine_text(x))
fake['text'] = fake['text'].apply(lambda x : combine_text(x))

In [104]:
#Temporary Save for Precaution

true.to_csv("True_all_treated.csv")
fake.to_csv("Fake_all_treated.csv")

## Splitting into train and test

In [106]:
true_test = true[20000:]
fake_test = fake[20000:]

true = true[:20000]
fake = fake[:20000]

In [107]:
#Check

true.shape[0] + true_test.shape[0],fake_test.shape[0] + fake.shape[0]

(21416, 23481)

In [108]:
combined_train = pd.concat([true, fake], axis = 'rows')
combined_test = pd.concat([true_test, fake_test], 'rows')

combined_train = combined_train.sample(frac = 1).reset_index(drop = True)
combined_test = combined_test.sample(frac = 1).reset_index(drop = True)

In [119]:
combined_train.to_csv('combined_train_treated.csv')
combined_test.to_csv('combined_test_treated.csv')

In [121]:
combined_train.iloc[7621]['text']

''